In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from random import shuffle
from tqdm import tqdm

TRAIN_DIR = '/Users/DBRSJ/Desktop/DeepDive_into_DeepLearning/Sentdex_kaggle/train'
TEST_DIR = '/Users/DBRSJ/Desktop/DeepDive_into_DeepLearning/Sentdex_kaggle/test'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}-model'.format(LR,'2conv-basic-video')

In [2]:
def label_img(img):
    word_label = img.split('.')[-3]
    if word_label == 'cat' : return [1,0]
    elif word_label == 'dog' : return [0,1]

In [3]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data
        

In [4]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path= os.listdir(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.resize(cv2.imread(path,cv2.IMREAD_GRAYSCALE),(IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img),img_num])
    np.save('test_data.npy',testing_data)
    return testing_data
    


In [5]:
train_data = create_train_data()

100%|██████████| 25000/25000 [01:15<00:00, 332.90it/s]


In [6]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation='relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')


/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [7]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded')

In [8]:
train = train_data[:-500]
test = train_data[-500:]

In [9]:
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]

In [30]:
model.fit({'input': X}, {'targets': Y}, n_epoch=5, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)


Training Step: 5382  | total loss: 12.23351 | time: 177.561s
| Adam | epoch: 021 | loss: 12.23351 - acc: 0.4687 -- iter: 24448/24500


Exception in thread Thread-89:
Traceback (most recent call last):
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/site-packages/tflearn/data_flow.py", line 187, in fill_feed_dict_queue
    data = self.retrieve_data(batch_ids)
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/site-packages/tflearn/data_flow.py", line 222, in retrieve_data
    utils.slice_array(self.feed_dict[key], batch_ids)
  File "/Users/DBRSJ/anaconda/envs/tf_env/lib/python3.6/site-packages/tflearn/utils.py", line 187, in slice_array
    return X[start]
IndexError: index 500 is out of bounds for axis 0 with size 500



KeyboardInterrupt: 

In [ ]:
tensorboard --logdir= foo:/Users/DBRSJ/Desktop/DeepDive_into_DeepLearning/Sentdex_kaggle/log 
    
    